In [ ]:
# Simple dataframe
import os
import pandas as pd

df = pd.read_csv("https://data.heatonresearch.com/data/t81-558/auto-mpg.csv")
print(df[0:5])

    mpg  cylinders  displacement horsepower  weight  acceleration  year  \
0  18.0          8         307.0        130    3504          12.0    70   
1  15.0          8         350.0        165    3693          11.5    70   
2  18.0          8         318.0        150    3436          11.0    70   
3  16.0          8         304.0        150    3433          12.0    70   
4  17.0          8         302.0        140    3449          10.5    70   

   origin                       name  
0       1  chevrolet chevelle malibu  
1       1          buick skylark 320  
2       1         plymouth satellite  
3       1              amc rebel sst  
4       1                ford torino  


## Missing Values

Missing values are a reality of machine learning.  Ideally, every row of data will have values for all columns.  However, this is rarely the case.  Most of the values are present in the MPG database.  However, there are missing values in the horsepower column.  A common practice is to replace missing values with the median value for that column.  The program calculates the median as described [here](https://www.mathsisfun.com/median.html).  The following code replaces any NA values in horsepower with the median:

In [ ]:
import os
import pandas as pd

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv", 
    na_values=['NA', '?'])
print(f"horsepower has na? {pd.isnull(df['horsepower']).values.any()}")
    
print("Filling missing values...")
med = df['horsepower'].median()
df['horsepower'] = df['horsepower'].fillna(med)
# df = df.dropna() # you can also simply drop NA values
                 
print(f"horsepower has na? {pd.isnull(df['horsepower']).values.any()}")

horsepower has na? True
Filling missing values...
horsepower has na? False


# Dealing with Outliers

Outliers are values that are unusually high or low.  Sometimes outliers are simply errors; this is a result of [observation error](https://en.wikipedia.org/wiki/Observational_error).  Outliers can also be truly large or small values that may be difficult to address. We typically consider outliers to be a value that is several standard deviations from the mean.  The following function can remove such values.    

In [ ]:
# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean())
                          >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)

The code below will drop every row from the Auto MPG dataset where the horsepower is more than two standard deviations above or below the mean.

In [ ]:
import pandas as pd
import os
import numpy as np
from sklearn import metrics
from scipy.stats import zscore

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv",
    na_values=['NA','?'])

# create feature vector
med = df['horsepower'].median()
df['horsepower'] = df['horsepower'].fillna(med)

# Drop the name column
df.drop('name',1,inplace=True)

# Drop outliers in horsepower
print("Length before MPG outliers dropped: {}".format(len(df)))
remove_outliers(df,'mpg',2)
print("Length after MPG outliers dropped: {}".format(len(df)))

pd.set_option('display.max_columns', 0)
pd.set_option('display.max_rows', 5)
display(df)

Length before MPG outliers dropped: 398
Length after MPG outliers dropped: 388


,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin
0,18.0,8,307.0,130.0,3504,12.0,70,1
1,15.0,8,350.0,165.0,3693,11.5,70,1
...,...,...,...,...,...,...,...,...
396,28.0,4,120.0,79.0,2625,18.6,82,1
397,31.0,4,119.0,82.0,2720,19.4,82,1


## Dropping Fields

Some fields are of no value to the neural network should be dropped.  The following code removes the name column from the MPG dataset.

In [ ]:
import os
import pandas as pd

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv",
    na_values=['NA','?'])

print(f"Before drop: {list(df.columns)}")
df.drop('name', 1, inplace=True)
print(f"After drop: {list(df.columns)}")

Before drop: ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin', 'name']
After drop: ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin']


## Concatenating Rows and Columns
Python can concatenate rows and columns together to form new data frames.  The code below creates a new data frame from the **name** and **horsepower** columns from the Auto MPG dataset.  The program does this by concatenating two columns together.

In [ ]:
# Create a new dataframe from name and horsepower

import os
import pandas as pd

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv",
    na_values=['NA','?'])

col_horsepower = df['horsepower']
col_name = df['name']
result = pd.concat([col_name, col_horsepower], axis=1)

pd.set_option('display.max_columns', 0)
pd.set_option('display.max_rows', 5)
display(result)

,name,horsepower
0,chevrolet chevelle malibu,130.0
1,buick skylark 320,165.0
...,...,...
396,ford ranger,79.0
397,chevy s-10,82.0


The **concat** function can also concatenate two rows together.  This code concatenates the first two rows and the last two rows of the Auto MPG dataset.

In [ ]:
# Create a new dataframe from first 2 rows and last 2 rows

import os
import pandas as pd

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv",
    na_values=['NA','?'])

result = pd.concat([df[0:2],df[-2:]], axis=0)

pd.set_option('display.max_columns', 7)
pd.set_option('display.max_rows', 0)
display(result)

,mpg,cylinders,displacement,...,year,origin,name
0,18.0,8,307.0,...,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,...,70,1,buick skylark 320
396,28.0,4,120.0,...,82,1,ford ranger
397,31.0,4,119.0,...,82,1,chevy s-10


## Training and Validation

We must evaluate a machine learning model based on its ability to predict data that it has never seen before.  Because of this, we often divide the training data into a validation and training set.  The machine learning model will learn from the training data, but ultimately be evaluated based on the validation data.

* **Training Data** - **In Sample Data** - The data that the neural network used to train. 
* **Validation Data** - **Out of Sample Data** - The data that the machine learning model is evaluated upon after it is fit to the training data.

There are two effective means of dealing with training and validation data:

* **Training/Validation Split** - The program splits the data according to some ratio between a training and validation (hold-out) set. Typical rates are 80% training and 20% validation.
* **K-Fold Cross Validation** - The program splits the data into several folds and models.  Because the program creates the same number of models as folds, the program can generate out-of-sample predictions for the entire dataset.

The code below performs a split of the MPG data into a training and validation set.  The training set uses 80% of the data, and the validation set uses 20%. Figure 2.TRN-VAL shows how a model is trained on 80% of the data and then validated against the remaining 20%.

**Figure 2.TRN-VAL: Training and Validation**
![Training and Validation](https://raw.githubusercontent.com/jeffheaton/t81_558_deep_learning/master/images/class_1_train_val.png "Training and Validation")


In [ ]:
import os
import pandas as pd
import numpy as np

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv",
    na_values=['NA','?'])

# Usually a good idea to shuffle
df = df.reindex(np.random.permutation(df.index)) 

mask = np.random.rand(len(df)) < 0.8
trainDF = pd.DataFrame(df[mask])
validationDF = pd.DataFrame(df[~mask])

print(f"Training DF: {len(trainDF)}")
print(f"Validation DF: {len(validationDF)}")

Training DF: 333
Validation DF: 65


## Saving a Dataframe to CSV

Many of the assignments in this course will require that you save a data frame to submit to the instructor.  The following code performs a shuffle and then saves a new copy.

In [ ]:
import os
import pandas as pd
import numpy as np

path = "."

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv",
    na_values=['NA','?'])

# Specify index = false to not write row numbers
df.to_csv("./auto-mpg-shuffle.csv", index=False) 
print("Done")

Done


## Saving a Dataframe to Pickle

A variety of software programs can make use of text files stored as CSV.  However, they do take longer to generate and can sometimes lose small amounts of precision in the conversion.  Another format is [Pickle](https://docs.python.org/3/library/pickle.html).  Generally, you will output to CSV because it is very compatible, even outside of Python.  The code below stores the Dataframe to Pickle.

In [ ]:
import os
import pandas as pd
import numpy as np
import pickle

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv",
    na_values=['NA','?'])

with open("./auto-mpg-shuffle.pkl","wb") as fp:
    pickle.dump(df, fp)

Loading the pickle file back into memory is accomplished by the following lines of code.  Notice that the index numbers are still jumbled from the previous shuffle?  Loading the CSV rebuilt (in the last step) did not preserve these values.

In [ ]:
import os
import pandas as pd
import numpy as np
import pickle

path = "."

df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/auto-mpg.csv",
    na_values=['NA','?'])

filename_read = os.path.join(path, "auto-mpg-shuffle.pkl")

with open(filename_write,"rb") as fp:
    df = pickle.load(fp)

pd.set_option('display.max_columns', 7)
pd.set_option('display.max_rows', 5)
display(df)

,mpg,cylinders,displacement,...,year,origin,name
387,38.0,6,262.0,...,82,1,oldsmobile cutlass ciera (diesel)
361,25.4,6,168.0,...,81,3,toyota cressida
...,...,...,...,...,...,...,...
358,31.6,4,120.0,...,81,3,mazda 626
237,30.5,4,98.0,...,77,1,chevrolet chevette


# Module 2 Assignment

You can find the first assignment here: [assignment 2](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/assignments/assignment_yourname_class2.ipynb)